In [1]:
import numpy as np

# Install HyperLogLog from https://github.com/svpcom/hyperloglog
import hyperloglog

In [2]:
# generate n key, value pairs where keys are integers in the range [0,r) and values are floats in [0,1) 
def generator(nr, r):
    for i in range(nr):
        np.random.seed(i) # the seed for reproducibility
        yield (np.random.randint(0, r), np.random.random())

In [3]:
gen = generator(10, 20)

In [4]:
for idx, val in gen:
    print(idx, val)

12 0.5928446182250183
5 0.9971848109388686
8 0.18508207817320688
10 0.07072488001099086
14 0.5472322491757223
3 0.055180123975368534
10 0.9474760765729829
15 0.22733907982646517
3 0.01111444062397371
1 0.21855867562607822


In [5]:
# compute the variance exactly by storing the values in a hash table
def compute_exact(gen):
    X = {}
    for idx, val in gen:
        X.setdefault(idx, 0)
        X[idx] += val 
    if len(X)< 10:
        print(X)
    s = 0
    s2 = 0
    for idx, val in X.items():
        s += val
        s2 += val**2
    mean = s/len(X)
    print("len X", len(X))
    print(s2)
    return s2/len(X) - mean**2

In [6]:
gen = generator(10, 5)
var = compute_exact(gen)

{4: 1.8746380026138718, 3: 0.7866190580415003, 0: 0.18508207817320688, 2: 1.918822412098645}
len X 4
7.8491720081512


In [7]:
# a small test comparing to numpy.std
var, np.std([1.8746380026138718, 0.7866190580415003, 0.18508207817320688, 1.918822412098645])**2

(0.5431202141356029, 0.5431202141356031)

In [8]:
# estimate the variance using count sketch
def estimate_variance(gen, cs_size, nr_sketches):
    """
    :param: gen: the number generator simulating a stream of updates (i, w_i)
    :param: cs_size: how many elements to keep in the count-sketch
    :param: nr_sketches: the median of how many estimates by count-sketches to compute
    """
    count_sketches = [[0 for _ in range(cs_size)] for _ in range(nr_sketches)] 
    # use a hyperloglog counter with 1% error to estimate the number of unique numbers
    hll = hyperloglog.HyperLogLog(0.01)     
    s = 0 
    for idx, val in gen:
        s += val
        hll.add(idx)
        for i in range(len(count_sketches)):
            count_sketch = count_sketches[i]
            np.random.seed(17*(i+1)*idx + 23)
            b = np.random.randint(0, cs_size)
            np.random.seed(43*(i+1)*idx+61)
            sign = 2*np.random.randint(0, 2)-1
            count_sketch[b] += sign*val
    s2_est = []
    for count_sketch in count_sketches:
        s2_est.append(np.dot(np.array(count_sketch), np.array(count_sketch)))
    s2 = np.median(s2_est)
    print(s2_est)
    print(s2)
    print("len hll", len(hll))
    mean = s/len(hll)
    return s2/len(hll) - mean**2
    

In [9]:
gen = generator(10, 5)
var = estimate_variance(gen, 10, 3)
print("Estimated varuance", var)

[7.849172008151199, 10.86793656485911, 4.899920048581352]
7.849172008151199
len hll 4
Estimated varuance 0.5431202141356033


In [10]:
gen = generator(1000000, 100000)
var_exact = compute_exact(gen)
print("Exact variance", var_exact)

len X 99997
2830748.2297524586
Exact variance 3.327018689526625


In [12]:
gen = generator(1000000, 100000)
var_est = estimate_variance(gen, 5000, 3)
print("Estimated variance", var_est)

[2857406.111330656, 2816147.092049377, 2714930.0952371154]
2816147.092049377
len hll 101422
Estimated variance 3.4823702605938784
